# MicroHapulator: Simulation Demo

**MicroHapulator** is software for forensic analysis of microhaplotype sequence data.
Features include the following:

- simulating simple (single-contributor) and complex (multi-contributor) DNA samples
- simulated MPS sequencing of user-specified microhap panels
- genotyping of DNA profiles from simple and complex DNA samples
- tools for deterministic and probabilistic interpretation of simple and complex samples

MicroHapulator relies on microhap marker definitions and allele frequencies from [MicroHapDB](https://github.com/bioforensics/MicroHapDB) and MPS error models included with [InSilicoSeq](https://github.com/HadrienG/InSilicoSeq/).

## Motivation

MPS sequence data from microhap loci of real DNA samples is not easily accessible due to a variety of factors, including privacy, intellectual property, cost, and the novelty of microhaplotypes as a forensic marker.
The development of MicroHapulator has thus benefited immensely from simulated genotypes and samples.
This notebook shows how mock data for two forensic scenarios was simulated.

> _**NOTE**: While most MicroHapulator commands require only a second or two to run, simulating MPS sequencing can take a few minutes per sample depending on the number of requested reads and the number of available processors._

## Scenario 1 — Simulating Genotypes

First we construct a hypothetical forensic scenario involving a reference sample obtained from a person of interest, as well as two evidentiary samples whose origin is unknown in the context of the investigation.

To fully construct this scenario we need to simulate genotype profiles for two individuals, **IND1** and **IND2**.
To do this, we must specify the population from which each parental haplotype originates and the panel of microhap markers that will be assayed.
We invoke the `mhpl8r sim` command to simulate each genotype.

In [1]:
mhpl8r sim --seed 24680 --out genotype-IND1.json Iberian Iberian beta-panel.txt

[MicroHapulator] running version 0.4.1
[MicroHapulator::sim] simulated microhaplotype variation at 50 markers
[MicroHapulator::sim] profile JSON written to genotype-IND1.json


Let's break down what each part of this command means.

- `mhl8r sim`: The command to simulate a genotype profile.
- `--seed 24680`: MicroHapulator uses a random number generator to sample alleles randomly at each marker for each parental haplotype. It's possible to set this random number generator to a predictable state using a specific "seed" value so that the same sequence of "random" numbers can be reproduced later. In other words, specifying a seed is indicating that I don't want *just any* random genotype, I want *that particular* random genotype. The seed value can be chosen arbitrarily since it has no other significance or meaning.
- `--out genotype-IND1.json`: MicroHapulator will save the simulated genotype in a file named `genotype-IND1.json`.
- `Iberian Iberian`: Use the "Iberian" population allele frequencies (ALFRED population "SA004108N") to sample alleles for both the maternal and paternal haplotypes. MicroHapulator can use allele frequency distributions from any population in the MicroHapDB database to simulate genotypes that realistically reflect true allele frequencies. MicroHapDB includes definitions for 290 microhaplotype markers and allele frequencies for 102 global populations and cohorts, which represents nearly all published microhap data.
- `beta-panel.txt`: A file containing identifiers for the 50 microhap markers we will assay in our hypothetical scenario.

We can peek at the top of this file to explore its contents.
As you can see, it contains marker names and genotypes in JSON (JavaScript Object Notation) format, which is easily consumed by computers and also human readable (if a bit verbose).
For example, this genotype is homozygous for the `T,G,G` allele at marker `mh01CP-016`, and heterzygous for the alleles `A,A,C,T` and `A,G,C,T` at marker `mh01KK-117`.
Here we can only see information for the first two markers, but information for all 50 markers is present in the remainder of the file.

In [2]:
head -n 25 genotype-IND1.json

{
    "markers": {
        "mh01CP-016": {
            "genotype": [
                {
                    "allele": "T,G,G",
                    "haplotype": 0
                },
                {
                    "allele": "T,G,G",
                    "haplotype": 1
                }
            ]
        },
        "mh01KK-117": {
            "genotype": [
                {
                    "allele": "A,A,C,T",
                    "haplotype": 0
                },
                {
                    "allele": "A,G,C,T",
                    "haplotype": 1
                }
            ]


Now let's simulate our second individual, storing its genotype in the file `genotype-IND2.json`.

In [3]:
mhpl8r sim --seed 13579 --out genotype-IND2.json Iberian Iberian beta-panel.txt

[MicroHapulator] running version 0.4.1
[MicroHapulator::sim] simulated microhaplotype variation at 50 markers
[MicroHapulator::sim] profile JSON written to genotype-IND2.json


## Scenario 1 — Simulated MPS Sequencing

Next we simulate Illumina MiSeq sequencing of three samples from these two individuals: a reference sample **REF1** and two evidentiary samples **EVD1** and **EVD2**.
Samples **REF1** and **EVD1** originate from individual **IND1**, and sample **EVD2** originates from individual **IND2**.
Of course, in a real case the identity of the reference sample donor would be known, but the identity of the evidentiary sample donor(s) would be unknown.

Let's start with sample **REF1**.

In [4]:
mhpl8r seq --num-reads 50000 --out reads-REF1.fastq.gz --seed 666410524 --threads 2 genotype-IND1.json

[MicroHapulator] running version 0.4.1
[MicroHapulator::seq] Individual seed=666410524 numreads=50000


Let's break the syntax of this command down.

- `mhpl8r seq`: The command to simulate MPS sequencing.
- `--num-reads 10000`: Simulate 10,000 reads.
- `--out reads-REF1.fastq.gz`: Save the reads in a file named `reads-REF1.fastq.gz`.
- `--seed 666410524`: Seed the random number generator. As with `mhpl8r sim`, running `mhpl8r seq` with the same random seed on the same genotype will produce exactly the same "random" reads. However, running `mhpl8r seq` on the same genotype twice using two different seeds represents sequencing two independent samples of the same individual.
- `--threads 2`: Accelerate the simulation using 2 threads. On more powerful computers, you may be able to use as many as 32 or 64 threads to accelerate the simulation.
- `genotype-IND1.json`: The file containing the genotype to be sequenced.

Now let's create sample **EVD1** from the same genotype.

In [5]:
mhpl8r seq --threads 2 --num-reads 5000 --seed 3374532379 --out reads-EVD1.fastq.gz genotype-IND1.json

[MicroHapulator] running version 0.4.1
[MicroHapulator::seq] Individual seed=3374532379 numreads=5000


And finally, sample **EVD2** from the genotype of individual **IND2**.

In [6]:
mhpl8r seq --threads 2 --num-reads 5000 --seed 3963949764 --out reads-EVD2.fastq.gz genotype-IND2.json

[MicroHapulator] running version 0.4.1
[MicroHapulator::seq] Individual seed=3963949764 numreads=5000


That concludes our simulation for Scenario 1.
We have "sequenced" three mock samples: **REF1**, **EVD1**, and **EVD2**.

In [7]:
ls -1 reads-REF1.fastq.gz reads-EVD1.fastq.gz reads-EVD2.fastq.gz

reads-EVD1.fastq.gz
reads-EVD2.fastq.gz
reads-REF1.fastq.gz


## Scenario 2

For our second scenario, we simulate a single evidentiary sample and three reference samples.
The evidentiary sample is a three-contributor mixture.
One of the reference samples matches a minor contributor to the evidentiary sample, while the other two reference samples originate from non-contributors.

Using the command syntax we discussed above, we start by simulating the genotypes of each involved individual.
We specify `SA004250L` for the population, which is ALFRED's identifier for European American population allele frequencies.

In [8]:
# Contributors
mhpl8r sim --seed 1234 --out genotype-CTRB1.json SA004250L SA004250L beta-panel.txt
mhpl8r sim --seed 5678 --out genotype-CTRB2.json SA004250L SA004250L beta-panel.txt
mhpl8r sim --seed 1029 --out genotype-CTRB3.json SA004250L SA004250L beta-panel.txt

# Non-contributors
mhpl8r sim --seed 3847 --out genotype-NON1.json SA004250L SA004250L beta-panel.txt
mhpl8r sim --seed 5656 --out genotype-NON2.json SA004250L SA004250L beta-panel.txt

[MicroHapulator] running version 0.4.1
[MicroHapulator::sim] simulated microhaplotype variation at 50 markers
[MicroHapulator::sim] profile JSON written to genotype-CTRB1.json
[MicroHapulator] running version 0.4.1
[MicroHapulator::sim] simulated microhaplotype variation at 50 markers
[MicroHapulator::sim] profile JSON written to genotype-CTRB2.json
[MicroHapulator] running version 0.4.1
[MicroHapulator::sim] simulated microhaplotype variation at 50 markers
[MicroHapulator::sim] profile JSON written to genotype-CTRB3.json
[MicroHapulator] running version 0.4.1
[MicroHapulator::sim] simulated microhaplotype variation at 50 markers
[MicroHapulator::sim] profile JSON written to genotype-NON1.json
[MicroHapulator] running version 0.4.1
[MicroHapulator::sim] simulated microhaplotype variation at 50 markers
[MicroHapulator::sim] profile JSON written to genotype-NON2.json


Invoking the `mhpl8r seq` command with a single genotype file, as we did above, will simulate sequencing of a single contributor sample.
Alternatively, we can simulate sequencing of a mixture sample by supplying multiple genotype files.
By default, MicroHapulator treats each genotype as an equal contributor, but it's possible to specify uneven contributions using the `--proportions` flag.
We will call this sample **EVD3** and store the corresponding reads in a file called `reads-EVD3.fastq.gz`.

In [9]:
# The \ at the end of the line means we're wrapping the command and continuing it on the following line
mhpl8r seq --proportions 0.7 0.25 0.05 --num-reads 50000 --threads 2 --out reads-EVD3.fastq.gz \
    genotype-CTRB1.json genotype-CTRB2.json genotype-CTRB3.json

[MicroHapulator] running version 0.4.1
[MicroHapulator::seq] Individual seed=2813709479 numreads=35000
[MicroHapulator::seq] Individual seed=3891776762 numreads=12500
[MicroHapulator::seq] Individual seed=2232112581 numreads=2500


Now we will construct our three reference samples.
Sample **REF2** will be derived from individual **CTRB2**, one of the minor contributors (25% of the reads) to the mixture sample **EVD3**.
Sample **REF3** and **REF4** will be derived from individuals **NON1** and **NON2**, non-contributors to the mixture.

In [10]:
mhpl8r seq --num-reads 50000 --threads 2 --out reads-REF2.fastq.gz genotype-CTRB2.json
mhpl8r seq --num-reads 50000 --threads 2 --out reads-REF3.fastq.gz genotype-NON1.json
mhpl8r seq --num-reads 50000 --threads 2 --out reads-REF4.fastq.gz genotype-NON2.json

[MicroHapulator] running version 0.4.1
[MicroHapulator::seq] Individual seed=4172766544 numreads=50000
[MicroHapulator] running version 0.4.1
[MicroHapulator::seq] Individual seed=3117232933 numreads=50000
[MicroHapulator] running version 0.4.1
[MicroHapulator::seq] Individual seed=2916178804 numreads=50000


That concludes our simulation for Scenario 2.
We "sequenced" four additional mock samples: **EVD3**, **REF2**, **REF3**, and **REF4**.
Visit the [demo-cli.ipynb](demo-cli.ipynb) notebook to see how MicroHapulator is used to analyze and interpret these samples in a forensic investigation.

In [11]:
ls -1 reads-EVD3.fastq.gz reads-REF2.fastq.gz reads-REF3.fastq.gz reads-REF4.fastq.gz

reads-EVD3.fastq.gz
reads-REF2.fastq.gz
reads-REF3.fastq.gz
reads-REF4.fastq.gz
